In [25]:
#Importing necessary libraries and frameworks
import tensorflow as tf
import scipy as sc
import numpy as np
import time
import sklearn.preprocessing
from scipy.signal import butter,lfilter
from tensorflow import keras


In [26]:
#Defining a butterworth filtter bandpass
def butter_bandpass(lowcut,highcut,fs,order=5):
    #Nyquist rate fs=2*fm
    #fm=0.5*fs
    nyq=0.5*fs
    lowcut=lowcut/nyq
    highcut=highcut/nyq
    #b,a=numerator and denominator values of the IIR Filter
    b,a=butter(order,[lowcut,highcut],btype='band')
    return b,a

In [27]:
#Defining one hot encoding
#This function is used to transfer one column label to one hot label
def one_hot(y_):
    #Function to encode output labels from number indexes
    #e.g [[5],[3],[0]] --> [[0,0,0,0,0,1],[0,0,0,1,0,0],[0,0,0,0,0,0]]
#     y_=y_.reshape((len(y_)))
    y_=np.array(y_)
    y_=y_.reshape(len(y_))
    n_values=np.max(y_)+1
    return np.eye(int(n_values))[np.array(y_,dtype=np.int32)]

In [28]:
#Checking
a1=[[5],[3],[2]]
a2=one_hot(a1)
print(a2)

[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]


In [29]:
def butter_bandpass_filter(data,lowcut,highcut,fs,order=5):
    b,a=butter_bandpass(lowcut,highcut,fs,order=order)
    y=lfilter(b,a,data)
    return y
    

Defining some variables used to load and process the EEG Data
1. **len_sample** = Length of each sample ( in seconds )
2. **full**= total number of samples per subject 
3. **n_fea**=number of features ( EEG Channels )
4. **label0-label7** = Defines the labels for each sample

In [30]:
len_sample=1
full=7000
len_a=int(full/len_sample)
label0=np.zeros(len_a)
label1=np.ones(len_a)
label2=np.ones(len_a)*2
label3=np.ones(len_a)*3
label4=np.ones(len_a)*4
label5=np.ones(len_a)*5
label6=np.ones(len_a)*6
label7=np.ones(len_a)*7
label=np.hstack((label0,label1,label2,label3,label4,label5,label6,label7))
label=label.T
label.shape=(len(label),1)
print(label)

[[0.]
 [0.]
 [0.]
 ...
 [7.]
 [7.]
 [7.]]


In [31]:
start1=time.process_time()

In [32]:
feature=sc.io.loadmat("EID-S.mat")
all=feature['eeg_close_8sub_1file']

In [33]:
n_fea=14
all=all[0:full*8,0:n_fea]

### Next steps - Performing a filtering operation on a EEG Dataset to extract the delta wave pattern using a Butterworth filter

1. **start2**= measures the starting time using `time.process_time()` function and stores it in a variable
2. Initialize an empty list **data_f1** to store the filtered EEG data
3. Loop over the columns of the `all` array using `range(all.shape[1])`
4. For each column, applies a bandpass filter using the butter_bandpass_filter function with the specified lowcut, highcut and sampling frequency values. The filtered data is then stored in the `data_f1 list`.
5. Convert the `data_f1` list to a numpy array and then transpose it
6. Replace the original `all` array with the filtered data array `data_f1`
7. Measure the ending time using the `time.process_time()` function and store it in a variable `time3`
8. Print the shape of the filtered data array and the elapsed timem for the filtering operation

In [34]:
#Timing the process
start=time.process_time()

#Empty list
data_f1=[]
for i in range(all.shape[1]):
    x=all[:,i]
    fs=128.0
    lowcut=0.5
    highcut=4.0
    y=butter_bandpass_filter(x,lowcut,highcut,fs,order=3)
    data_f1.append(y)

In [35]:
data_f1=np.array(data_f1)
data_f1=data_f1.T
print('data_f1.shape',data_f1.shape)

data_f1.shape (56000, 14)


In [36]:
all=data_f1
end=time.process_time()
print('PD time',start1-end)
all=np.hstack((all,label))
print(all.shape)

PD time -0.015625
(56000, 15)


In [37]:
#all=combine data
data_size=all.shape[0]

feature_all=all[:,0:n_fea]
print(all[:,n_fea:n_fea+1])

[[0.]
 [0.]
 [0.]
 ...
 [7.]
 [7.]
 [7.]]


In [38]:
feature_all=feature_all-4200 #minus Direct Current
#z-score scaling
feature_all=sklearn.preprocessing.scale(feature_all)

In [39]:
label_all=all[:,n_fea:n_fea+1]
all=np.hstack((feature_all,label_all))
print(all.shape)

(56000, 15)


In [40]:
#Using the first subject as testing subject
np.random.shuffle(all)

In [41]:
train_data=all[0:int(data_size*0.875)] #Training data is 87.5% of original
test_data=all[int(data_size*0.875):data_size]

no_fea=n_fea
n_steps=len_sample

feature_training=train_data[:,0:no_fea]
feature_training=feature_training.reshape([train_data.shape[0],n_steps,no_fea])

feature_testing=test_data[:,0:no_fea]
feature_training = feature_training.reshape([train_data.shape[0], n_steps, no_fea])


feature_testing = test_data[:,0:no_fea]

feature_testing = feature_testing.reshape([test_data.shape[0], n_steps, no_fea])

In [42]:
label_training=train_data[:,no_fea]
label_training=one_hot(label_training)
label_testing=test_data[:,no_fea]
print(label_testing)

[7. 1. 3. ... 3. 0. 1.]


In [43]:
print(all.shape)

(56000, 15)


### Batch Splitting
**Splitting the input data into batches for training in order to process it more efficiently**

***Steps Involved***
1. It calculates the batch size by multiplying the total data size by 0.125 (12.5%).
2. It creates an empty list called train_fea.
3. It sets the number of groups to 7.
4. It loops through each group and selects the corresponding slice of the feature_training array to add to the train_fea list. Each slice is determined by multiplying the batch size by the group index and adding 0 to the start and end indices for the first group, adding one batch size to both the start and end indices for the second group, and so on. The result is that train_fea is a list of 7 arrays, each with batch_size rows and some number of columns.
5. It prints the shape of the first element of train_fea (which should be (batch_size, num_features)) and the elements of rows 235 and 236 of the first element of train_fea.
6. It creates an empty list called train_label.
7. It loops through each group and selects the corresponding slice of the label_training array to add to the train_label list. The slices are determined in the same way as for train_fea. The result is that train_label is a list of 7 arrays, each with batch_size rows and some number of columns.
8. It prints the shape of the first element of train_label (which should be (batch_size, num_classes)). 


In [44]:
a=feature_training
b=feature_testing
nodes=30
lameda=0.001
lr=0.001

batch_size=int(data_size*0.125)

n_group=7

def range_calculator(listname,listdest):
    for i in range(n_group):
        f=listname[(0+batch_size*i):(batch_size+batch_size*i)]
        listdest.append(f)

train_fea=[]
train_label=[]

range_calculator(a,train_fea)
range_calculator(label_training,train_label)

print(train_fea[0].shape)
print(train_fea[0][235:237])

print(train_label[0].shape)

(7000, 1, 14)
[[[ 0.08028771 -0.22681946  0.05469709  0.02443724 -0.002127
    0.04619136  0.05424574 -0.02418474 -0.12383184 -0.01481329
   -0.04651189  0.06176732 -0.13170864  0.04388825]]

 [[-0.30657352 -0.51793876 -0.16363107 -0.33376376 -0.12930287
   -0.28284165 -0.24096585 -0.31190144 -0.24371498 -0.20759036
   -0.08147497 -0.21883572 -0.13740341 -0.27573951]]]
(7000, 8)


### Defining an RNN Cell 

In [45]:
n_inputs = no_fea  # MNIST data input (img shape: 11*99)
n_hidden1_units = nodes   # neurons in hidden layer
n_hidden2_units = nodes
n_hidden3_units = nodes
n_hidden4_units = nodes
n_classes = 8  # MNIST classes (0-9 digits)
batch_size = 128
n_steps = None

# Define input
inputs = keras.Input(shape=(n_steps, n_inputs), name='input')

# Define layers
flatten = keras.layers.Flatten()(inputs)

dense1 = keras.layers.Dense(n_hidden1_units, activation='sigmoid', name='dense1')(flatten)
dense2 = keras.layers.Dense(n_hidden2_units, name='dense2')(dense1)
dense3 = keras.layers.Dense(n_hidden3_units, name='dense3')(dense2)
dense4 = keras.layers.Dense(n_hidden4_units, name='dense4')(dense3)

# Define LSTM cell
lstm_cell_1 = keras.layers.LSTM(n_hidden3_units, return_sequences=True, return_state=True, name='lstm1')
outputs, state_h, state_c = lstm_cell_1(dense4)

# Define attention model
dense_att1 = keras.layers.Dense(n_hidden4_units, activation='tanh', name='dense_att1')(outputs)
dense_att2 = keras.layers.Dense(1, activation='softmax', name='dense_att2')(dense_att1)
outputs_att = keras.layers.Dot(axes=1, name='dot')([dense_att2, outputs])

# Define output layer
dense_out = keras.layers.Dense(n_classes, name='dense_out')(outputs_att)

# Define model
model = keras.Model(inputs=inputs, outputs=dense_out, name='rnn_model')

# Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()

ValueError: The last dimension of the inputs to `Dense` should be defined. Found `None`.

In [ ]:
#hyperparameters
n_inputs=no_fea
n_hidden1_units=nodes
n_hidden2_units=nodes
n_hidden3_units=nodes
n_hidden4_units=nodes
n_classes=8

#tf graph input 
x=tf.keras.layers.Input(shape=(None,n_inputs),name="x",dtype='float32')
y=tf.keras.layers.Input(shape=(n_classes),dtype='float32',name="y")

In [ ]:
#Defining weights
weights = {
    'in': tf.Variable(tf.random.normal([n_inputs, n_hidden1_units]), trainable=True),
    'a': tf.Variable(tf.random.normal([n_hidden1_units, n_hidden1_units]), trainable=True),
    'hidd2': tf.Variable(tf.random.normal([n_hidden1_units, n_hidden2_units])),
    'hidd3': tf.Variable(tf.random.normal([n_hidden2_units, n_hidden3_units])),
    'hidd4': tf.Variable(tf.random.normal([n_hidden3_units, n_hidden4_units])),
    'out': tf.Variable(tf.random.normal([n_hidden4_units, n_classes]), trainable=True),
    'att': tf.Variable(tf.random.normal([n_inputs, n_hidden4_units]), trainable=True),
    'att2': tf.Variable(tf.random.normal([1, batch_size]), trainable=True),
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden1_units])),
    'hidd2': tf.Variable(tf.constant(0.1, shape=[n_hidden2_units])),
    'hidd3': tf.Variable(tf.constant(0.1, shape=[n_hidden3_units])),
    'hidd4': tf.Variable(tf.constant(0.1, shape=[n_hidden4_units])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes]), trainable=True),
    'att': tf.Variable(tf.constant(0.1, shape=[n_hidden4_units])),
    'att2': tf.Variable(tf.constant(0.1, shape=[n_hidden4_units])),
}

In [ ]:
def RNN(X,weights,biases):
    #hidden layer for input to cell
    ##############################
    
    X=tf.reshape(X,[-1,n_inputs])
    
    #hidden layer
    X_hidd1=tf.sigmoid(tf.matmul(X,weights['in'])+biases['in'])
    X_hidd2=tf.matmul(X_hidd1,weights['hidd2'])+biases['hidd2']
    X_hidd3=tf.matmul(X_hidd2,weights['hidd3'])+biases['hidd3']
    
    X_in=tf.reshape(X_hidd3,[-1,n_steps,n_hidden4_units])
    
    
    #cell
    #####################################
    
    #basic LSTM Cell
    lstm_cell_1=tf.keras.layers.LSTMCell(n_hidden3_units,unit_forget_bias=True)
    init_state=lstm_cell_1.get_initial_state(batch_size=batch_size,dtype=tf.float32)
    
    outputs,final_state=tf.keras.layers.RNN(lstm_cell_1,return_sequences=True,return_state=True)(X_in,initial_state=init_state)
    
    #outputs
    outputs=tf.unstack(tf.transpose(outputs,[1,0,2])) #states is the last output
    
    #attention based model
    X_att2=final_state[0] #weights
    outputs_att=tf.multiply(outputs[-1],X_att2)
    results=tf.matmul(outputs_att,weights['out'])+biases['out']
    
    return results,outputs[-1]


pred,Feature,_=RNN(x,weights,biases)
    

In [ ]:
lamena=lameda
#Computing l2 loss
l2=lamena*sum(tf.nn.l2_loss(tf_var) for tf_var in tf.compat.v1.trainable_variables())
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))+l2

In [ ]:

# tf.scalar_summary('loss', cost)

lr=lr
train_op = tf.train.AdamOptimizer(lr).minimize(cost)
pred_result =tf.argmax(pred, 1,name="pred_result")
label_true =tf.argmax(y, 1)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
rnn_s = time.clock()
with tf.Session(config=config) as sess:
    sess.run(init)
    step = 0
    while step < 2000:
        for i in range(n_group):
            sess.run(train_op, feed_dict={
                x: train_fea[i],
                y: train_label[i],
                })
        #  early stopping
        if sess.run(accuracy, feed_dict={x: b,y: label_testing,})>0.999:
            print(
            "The lamda is :", lamena, ", Learning rate:", lr, ", The step is:", step, ", The accuracy is: ",
            sess.run(accuracy, feed_dict={
                x: b,
                y: label_testing,
            }))
            break
        if step % 10 == 0:
            pp=sess.run(pred_result,feed_dict={x: b, y: label_testing})
            print "predict",pp[0:10]
            gt=np.argmax(label_testing, 1)
            print "groundtruth", gt[0:10]
            hh = sess.run(accuracy, feed_dict={
                x: b,
                y: label_testing,
            })
            h2=sess.run(accuracy,  feed_dict={x: train_fea[i],
                y: train_label[i]})
            print "training acc", h2
            print("The lamda is :",lamena,", Learning rate:",lr,", The step is:",step,
                  ", The accuracy is:", hh,", The train accuracy is:", h2)
            print("The cost is :",sess.run(cost, feed_dict={
                x: b,
                y: label_testing,
            }))
        step += 1

    endtime=time.clock()
    B=sess.run(Feature, feed_dict={
            x: train_fea[0],
            y: train_label[0],
        })
    for i in range(1,n_group):
        D=sess.run(Feature, feed_dict={
            x: train_fea[i],
            y: train_label[i],
        })
        B=np.vstack((B,D))
    B = np.array(B)
    print B.shape
    Data_train = B  # Extracted deep features
    Data_test = sess.run(Feature, feed_dict={x: b, y: label_testing})
print("RNN run time:", endtime-rnn_s)

# XGBoost
xgb_s=time.clock()
xg_train = xgb.DMatrix(Data_train, label=np.argmax(label_training,1))
xg_test = xgb.DMatrix(Data_test, label=np.argmax(label_testing,1))

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob' # can I replace softmax by SVM??
# softprob produce a matrix with probability value of each class
# scale weight of positive examples
param['eta'] = 0.7

param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['subsample']=0.9
param['num_class'] =n_classes


np.set_printoptions(threshold=np.nan)
watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 500
bst = xgb.train(param, xg_train, num_round, watchlist );
time8=time.clock()
pred = bst.predict(xg_test) ;
xgb_e=time.clock()
print('xgb run time', xgb_e -xgb_s)
print('RNN acc', hh)
